# Agenda

1. The iterator protocol
    - Adding iteration to your classes
    - Different techniques for that
2. Generator functions
3. Generator comprehensions
4. Decorators
5. Threading and multiprocessing 

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
        
p1 = Person('name1')        
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [2]:
import weakref

In [3]:
p3 = weakref.ref(p2)


In [4]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [5]:
p3.ref

AttributeError: 'weakref' object has no attribute 'ref'

In [7]:
w = weakref.ref(Person('name10'))

In [8]:
w

<weakref at 0x10b2aabd0; dead>

In [9]:
type(w)

weakref

In [10]:
dir(w)

['__call__',
 '__callback__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [11]:
p3 = weakref.ref(p2)


In [13]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [14]:
p1 = Person('name1')
p2 = Person('name2')

all_people = {1: p1, 2: p2}

In [15]:
del(p1)

In [16]:
len(all_people)

2

In [17]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [18]:
all_people

In [20]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [21]:
len(all_people)

2

In [22]:
del(p1)

In [23]:
len(all_people)

2

In [24]:
len(all_people)

2

In [25]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [30]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [31]:
del(p1)

In [32]:
len(all_people)

1

In [33]:
del(p2)

In [34]:
len(all_people)

0

# Iterator protocol

In [35]:
s = 'abcde'

for one_character in s:
    print(one_character)

a
b
c
d
e


# Protocol

1. Ask an object if it's iterable (`iter`)
    - Returns an iterator object if it is iterable
    - Raises an exception if it's not iterable
2. Ask the iterator we got back for its next item (`next`)
3. Each time we call `next`, we'll get an object
4. When we reach the end of the iteration, we get... the `StopIteration` exception

In [36]:
iter(s)

In [37]:
iter(s)

In [38]:
iter(s)

In [39]:
i = iter(s)  # store the string iterator in "i"

In [40]:
next(i)

'a'

In [41]:
next(i)

'b'

In [42]:
next(i)

'c'

In [43]:
next(i)

'd'

In [44]:
next(i)

'e'

In [45]:
next(i)

StopIteration: 

In [46]:
for i in 10:
    print(i)

TypeError: 'int' object is not iterable

In [47]:
iter(10)

TypeError: 'int' object is not iterable

In [55]:
class MyIterator:
    def __init__(self, data):
        print(f'Now in MyIterator.__init__ with {data=}')
        self.data = data
        self.index = 0
        
    def __iter__(self):
        print(f'Now in MyIterator.__iter__')
        return self   # the object is its own iterator!
    
    def __next__(self):
        print(f'Now entering MyIterator.__next__')
        if self.index >= len(self.data):
            print(f'\t{self.index=}, ending the loop')
            raise StopIteration
            
        value = self.data[self.index]
        print(f'\t{self.index=}, {value=}')
        self.index += 1
        return value

m = MyIterator('abc')

for one_item in m:
    print(one_item)

Now in MyIterator.__init__ with data='abc'
Now in MyIterator.__iter__
Now entering MyIterator.__next__
	self.index=0, value='a'
a
Now entering MyIterator.__next__
	self.index=1, value='b'
b
Now entering MyIterator.__next__
	self.index=2, value='c'
c
Now entering MyIterator.__next__
	self.index=3, ending the loop


In [51]:
f = open('/etc/passwd')

iter(f) is f

True

# Exercise: Circle

1. Create a class, `Circle`, which takes two arguments:
    - an iterable piece of data (`data`)
    - an integer (`maxtimes`)
2. If we run a `for` loop on an instance of `Circle`, we'll get `maxtimes` results back.
3. The return values will come from `data`.
    - If the number of elements in `data` is larger than `maxtimes`, then we'll just end after `maxtimes`, producing one value at a time.
    - If the number of elements in `data` is smaller than `maxtimes`, then when we get to the end of the data, we'll come back to index 0 -- thus going around and around until we give the number of results indicated in `maxtimes`.
    
```python
c = Circle('abc', 7)

for one_item in c:
    print(one_item)
```